<a href="https://colab.research.google.com/github/suriarasai/BEAD2024/blob/main/colab/FlightDelays_using_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resilient Distributed Data Set
Resilient Distributed Datasets (RDDs) are collections of immutable JVM objects that are distributed across an Apache Spark cluster. An RDD is the fundamental dataset type of Apache Spark; any action on a Spark DataFrame eventually gets translated into a highly optimized execution of transformations and actions on RDDs.

## Setup PySpark
Spark requires a handful of environments to be present on the machine before we can use it. The below codes help to install pyspark and related tools.

In [2]:
# install pyspark using pip
!pip install --ignore-install -q pyspark
# install findspark using pip
!pip install --ignore-install -q findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 15.5 MB/s eta 0:00:00


## Spark Session
The below codes are used to create a Spark session object, and also set up related UI port.

In [3]:
#from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
import collections
spark = SparkSession.builder.master("local").appName("Flight Delay").config('spark.ui.port', '4050').getOrCreate()


## Creating RDD
There are two ways to create an RDD in PySpark: we can either use the parallelize() method—a collection (list or an array of some elements) or reference a file (or files) located either locally or through an external source, as noted in subsequent recipes.

The following code snippet creates RDD (myRDD) using the sc.parallelize() method:

In [ ]:
myRDD = spark.sparkContext.parallelize([('Suria', 21), ('Venkat', 18), ('Liu Fan',16), ('Bob', 18), ('Scott', 17)])
myRDD.take(5)

[('Suria', 21), ('Venkat', 18), ('Liu Fan', 16), ('Bob', 18), ('Scott', 17)]


Reading data from a file involves file besing stored locally or in hadoop or AWS S3 or Azure WASB or Google Cloud Storage or Data Bricks storage. Examples for the same are provided below:
1. sc.textFile('/local folder/filename.csv')
2. sc.textFile('hdfs://folder/filename.csv')
3. sc.textFile('s3://bucket/folder/filename.csv')
4. sc.textFile('wasb://bucket/folder/filename.csv')
5. sc.textFile('gs://bucket/folder/filename.csv')
6. sc.textFile('dbfs://folder/filename.csv')

For this workshop, please use the data from GIT two files (airport-codes-na.txt and departuredelays.csv).

Mount them into your google drive using the below file upload button.

Note that the files can be copied from this [URL](https://github.com/suriarasai/BEAD2024/tree/main/data).



In [ ]:
from google.colab import files
uploaded = files.upload()

Saving airport-codes-na.txt to airport-codes-na.txt


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Alternatively, you can also upload the folders in your mounted google drive as shown below.

In [8]:
airportRDD = spark.sparkContext.textFile("/content/drive/MyDrive/data/airport-data/airport-codes-na.txt").map(lambda element: element.split("\t"))



In [9]:
airportRDD.take(5)


[['City', 'State', 'Country', 'IATA'],
 ['Abbotsford', 'BC', 'Canada', 'YXX'],
 ['Aberdeen', 'SD', 'USA', 'ABR'],
 ['Abilene', 'TX', 'USA', 'ABI'],
 ['Akron', 'OH', 'USA', 'CAK']]

In [ ]:
airportRDD.count()


527

In [10]:
airportRDD.getNumPartitions()

1

In [11]:
# Setup the RDD: flights
flights = (
    spark.sparkContext
    .textFile('/content/drive/MyDrive/data/airport-data/departuredelays.csv')
    .map(lambda element: element.split(","))
)

In [12]:
flights.take(5)

[['date', 'delay', 'distance', 'origin', 'destination'],
 ['01011245', '6', '602', 'ABE', 'ATL'],
 ['01020600', '-8', '369', 'ABE', 'DTW'],
 ['01021245', '-2', '602', 'ABE', 'ATL'],
 ['01020605', '-4', '602', 'ABE', 'ATL']]

In [14]:
# User filter() to filter where second column == "WA"
(
    airportRDD
    .map(lambda c: (c[0], c[1]))
    .filter(lambda c: c[1] == "WA")
    .take(5)
)

[('Bellingham', 'WA'),
 ('Moses Lake', 'WA'),
 ('Pasco', 'WA'),
 ('Pullman', 'WA'),
 ('Seattle', 'WA')]

In [15]:
# Filter only second column == "WA",
# select first two columns within the RDD,
# and flatten out all values
(
    airportRDD
    .filter(lambda c: c[1] == "WA")
    .map(lambda c: (c[0], c[1]))
    .flatMap(lambda x: x)
    .take(10)
)

['Bellingham',
 'WA',
 'Moses Lake',
 'WA',
 'Pasco',
 'WA',
 'Pullman',
 'WA',
 'Seattle',
 'WA']

In [ ]:
# Provide the distinct elements for the
# third column of airports representing
# countries
(
    airportRDD
    .map(lambda c: c[2])
    .distinct()
    .take(5)
)

In [ ]:
# Provide a sample based on 0.001% the
# flights RDD data specific to the fourth
# column (origin city of flight)
# without replacement (False) using random
# seed of 123
(
    flights
    .map(lambda c: c[3])
    .sample(False, 0.001, 123)
    .take(5)
)

['ABQ', 'AEX', 'AGS', 'ANC', 'ATL']

In [18]:
# Flights data
#  e.g. (u'JFK', u'01010900')
flt = flights.map(lambda c: (c[3], c[0]))

# Airports data
# e.g. (u'JFK', u'NY')
air = airportRDD.map(lambda c: (c[3], c[1]))

# Execute inner join between RDDs
flt.join(air).take(5)

[('ABE', ('01011245', 'PA')),
 ('ABE', ('01020600', 'PA')),
 ('ABE', ('01021245', 'PA')),
 ('ABE', ('01020605', 'PA')),
 ('ABE', ('01031245', 'PA'))]

In [19]:
# Let's re-partition this to 8 so we can have 8
# partitions
flights2 = flights.repartition(8)

# Checking the number of partitions for the flights2 RDD
flights2.getNumPartitions()


8

In [20]:
# View each row within RDD + the index
# i.e. output is in form ([row], idx)
ac = airportRDD.map(lambda c: (c[0], c[3]))
ac.zipWithIndex().take(5)

[(('City', 'IATA'), 0),
 (('Abbotsford', 'YXX'), 1),
 (('Aberdeen', 'ABR'), 2),
 (('Abilene', 'ABI'), 3),
 (('Akron', 'CAK'), 4)]

In [21]:
# Create `a` RDD of Washington airports
a = (
    airportRDD
    .zipWithIndex()
    .filter(lambda row, idx : idx > 0)
    .map(lambda row, idx: row)
    .filter(lambda c: c[1] == "WA")
)

# Create `b` RDD of British Columbia airports
b = (
    airportRDD
    .zipWithIndex()
    .filter(lambda row, idx: idx > 0)
    .map(lambda row, idx: row)
    .filter(lambda c: c[1] == "BC")
)

# Union WA and BC airports
a.union(b)


UnionRDD[42] at union at NativeMethodAccessorImpl.java:0

In [22]:
# Flights data
#  e.g. (u'JFK', u'01010900')
flt = flights.map(lambda c: (c[3], c[0]))

# Airports data
# e.g. (u'JFK', u'NY')
air = airportRDD.map(lambda c: (c[3], c[1]))

# Execute inner join between RDDs
flt.join(air).take(5)

[('ABE', ('01011245', 'PA')),
 ('ABE', ('01020600', 'PA')),
 ('ABE', ('01021245', 'PA')),
 ('ABE', ('01020605', 'PA')),
 ('ABE', ('01031245', 'PA'))]

In [24]:
# Setup the RDD: airports
airports = (
    spark.sparkContext
    .textFile('/content/drive/MyDrive/data/airport-data/airport-codes-na.txt')
    .map(lambda element: element.split("\t"))
)

airports.take(5)

[['City', 'State', 'Country', 'IATA'],
 ['Abbotsford', 'BC', 'Canada', 'YXX'],
 ['Aberdeen', 'SD', 'USA', 'ABR'],
 ['Abilene', 'TX', 'USA', 'ABI'],
 ['Akron', 'OH', 'USA', 'CAK']]

In [ ]:
# Setup the RDD: flights
flights2 = (
    spark.sparkContext
     .textFile('/content/drive/MyDrive/data/airport-data/departuredelays.csv', minPartitions=8)
    .map(lambda line: line.split(","))
)

flights2.take(5)


[['date', 'delay', 'distance', 'origin', 'destination'],
 ['01011245', '6', '602', 'ABE', 'ATL'],
 ['01020600', '-8', '369', 'ABE', 'DTW'],
 ['01021245', '-2', '602', 'ABE', 'ATL'],
 ['01020605', '-4', '602', 'ABE', 'ATL']]

In [ ]:
# Print to console the first 3 elements of
# the airports RDD
airportRDD.take(3)

[['City', 'State', 'Country', 'IATA'],
 ['Abbotsford', 'BC', 'Canada', 'YXX'],
 ['Aberdeen', 'SD', 'USA', 'ABR']]

In [ ]:
# Return all airports elements
# filtered by WA state
airportRDD.filter(lambda c: c[1] == "WA").collect()

[['Bellingham', 'WA', 'USA', 'BLI'],
 ['Moses Lake', 'WA', 'USA', 'MWH'],
 ['Pasco', 'WA', 'USA', 'PSC'],
 ['Pullman', 'WA', 'USA', 'PUW'],
 ['Seattle', 'WA', 'USA', 'SEA'],
 ['Spokane', 'WA', 'USA', 'GEG'],
 ['Walla Walla', 'WA', 'USA', 'ALW'],
 ['Wenatchee', 'WA', 'USA', 'EAT'],
 ['Yakima', 'WA', 'USA', 'YKM']]

In [ ]:
# Return all airports elements
# filtered by WA state
airportRDD.filter(lambda c: c[1] == "WA").collect()

[['Bellingham', 'WA', 'USA', 'BLI'],
 ['Moses Lake', 'WA', 'USA', 'MWH'],
 ['Pasco', 'WA', 'USA', 'PSC'],
 ['Pullman', 'WA', 'USA', 'PUW'],
 ['Seattle', 'WA', 'USA', 'SEA'],
 ['Spokane', 'WA', 'USA', 'GEG'],
 ['Walla Walla', 'WA', 'USA', 'ALW'],
 ['Wenatchee', 'WA', 'USA', 'EAT'],
 ['Yakima', 'WA', 'USA', 'YKM']]

In [ ]:
# Calculate the total delays of flights
# between SEA (origin) and SFO (dest),
# convert delays column to int
# and summarize
flights\
 .filter(lambda c: c[3] == 'SEA' and c[4] == 'SFO')\
 .map(lambda c: int(c[1]))\
 .reduce(lambda x, y: x + y)

22293

Workshop Ends Here